In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
ner_feature = raw_datasets["train"].features["ner_tags"]
print(ner_feature)
pos_feature = raw_datasets["train"].features["pos_tags"]
print(pos_feature)
chunk_feature = raw_datasets["train"].features["chunk_tags"]
print(chunk_feature)
ner_label_names = ner_feature.feature.names
print(ner_label_names)
pos_label_names = pos_feature.feature.names
print(pos_label_names)
chunk_label_names = pos_feature.feature.names
print(chunk_label_names)

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)
Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)
Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 

In [8]:
n = 5
words = raw_datasets["train"][n]["tokens"]
ner_labels = raw_datasets["train"][n]["ner_tags"]
pos_labels = raw_datasets["train"][n]["pos_tags"]
chunk_labels = raw_datasets["train"][n]["chunk_tags"]

line1 = ""
line2 = ""
line3 = ""
line4 = ""

for word, ner_label, pos_label, chunk_label in zip(words, ner_labels, pos_labels, chunk_labels):
    ner_full_label = ner_label_names[ner_label]
    pos_full_label = pos_label_names[pos_label]
    chunk_full_label = chunk_label_names[chunk_label]
    
    max_length = max(
        [len(word), len(ner_full_label), len(pos_full_label), len(chunk_full_label)]
    )
    
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += ner_full_label + " " * (max_length - len(ner_full_label) + 1)
    line3 += pos_full_label + " " * (max_length - len(pos_full_label) + 1)
    line4 += chunk_full_label + " " * (max_length - len(chunk_full_label) + 1)

print(line1)
print(line2)
print(line3)
print(line4)

" We  do  n't support any such recommendation because we  do  n't see any grounds for it  , " the Commission 's  chief spokesman Nikolaus van   der   Pas   told a  news briefing . 
O O   O   O   O       O   O    O              O       O   O   O   O   O   O       O   O   O O O   B-ORG      O   O     O         B-PER    I-PER I-PER I-PER O    O  O    O        O 
" PRP VBP RB  VB      DT  JJ   NN             IN      PRP VBP RB  VB  DT  NNS     IN  PRP , " DT  NNP        POS JJ    NN        NNP      NNP   FW    NNP   VBD  DT NN   NN       . 
" CD  NN  NNP NNP     CD  DT   DT             JJR     CD  NN  NNP NNP CD  DT      EX  CD  " " CD  DT         CD  DT    DT        DT       DT    DT    DT    NN   CD DT   DT       " 


In [9]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [10]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [11]:
def align_labels_first_subtoken_only(labels, word_ids):
    new_labels = []
    current_word = None
    first_subtoken = True

    for word_id in word_ids:
        if word_id is None:
            # Special tokens
            new_labels.append(-100)
            current_word = None
            first_subtoken = True

        else:
            if word_id != current_word:
                # New word → assign real label
                current_word = word_id
                first_subtoken = True
                new_labels.append(labels[word_id])
            else:
                # Same word → give -100 to all other subtokens
                if first_subtoken:
                    new_labels.append(labels[word_id])
                else:
                    new_labels.append(-100)
                first_subtoken = False

    return new_labels


In [12]:
print(raw_datasets["train"][5]["tokens"])
labels = raw_datasets["train"][5]["pos_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_first_subtoken_only(labels, word_ids))

['"', 'We', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"', 'the', 'Commission', "'s", 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '.']
[0, 28, 41, 30, 37, 12, 16, 21, 15, 28, 41, 30, 37, 12, 24, 15, 28, 6, 0, 12, 22, 27, 16, 21, 22, 22, 14, 22, 38, 12, 21, 21, 7]
[-100, 0, 28, 41, 30, 37, 12, 16, 21, 21, 15, -100]


In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_first_subtoken_only(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [14]:
raw_datasets["train"].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [17]:
for i in range(10):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]
[-100, 5, 5, -100, -100, 0, 0, -100, -100, -100, -100]
[-100, 0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 2, -100, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, 0, 0, 0, 0, 0, -100]
[-100, 1, 1, -100, 0, 7, 7, -100, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 7, 7, -100, 8, 8, -100, -100, 8, 8, -100, -100, -100, 0, 7, 7, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 1, 1, -100, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
import evaluate

metric = evaluate.load("seqeval")

In [26]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [ner_label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [28]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [29]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

C:\Users\rjdis\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rjdis\.cache\huggingface\hub\datasets--glue. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTT

In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [35]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.57.3
C:\Users\rjdis\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\__init__.py
